In [ ]:
from analyze_modes import *

In [ ]:
CLINICAL_DATA_XL = "/Users/inad001/Documents/SSCP25/Data and scripts SSCP25/utils/ClinicalData_OUS_MADPatients_EIVIND_29_4_2021.xlsx"
PCA_SCORES_CSV = "./PCA_results/pca_scores.csv"
NUM_MODES = 10

run_analysis_arrhythmia(CLINICAL_DATA_XL, PCA_SCORES_CSV, NUM_MODES)

In [ ]:
run_analysis_fibrosis(CLINICAL_DATA_XL, PCA_SCORES_CSV, NUM_MODES)

In [ ]:
run_analysis_mad(CLINICAL_DATA_XL, PCA_SCORES_CSV, NUM_MODES)